In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

In [ ]:
cards = pd.read_csv('mythic.csv')
names = list(cards['name'])
links = list(cards['link'])

In [ ]:
conn = sqlite3.connect('mtg.db')
c = conn.cursor()
# c.execute("DROP TABLE card_prices")
#c.execute('''CREATE TABLE card_prices (card text, price_obs text, UNIQUE(card, price_obs))''')
#conn.commit()

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
def get_data(card, link):
    driver.get(link)
    time.sleep(20)
    chart = driver.find_element_by_class_name('highcharts-plot-background')
    chart.size['width']
    results = []
    h1 = chart.size['height'] - 20
    h2 = h1 + 1
    w = chart.size['width']
    for i in range(0, w):
        action = webdriver.common.action_chains.ActionChains(driver)
        action.move_to_element_with_offset(chart, i, h1)
        action.move_to_element_with_offset(chart, i, h2)
        action.move_to_element_with_offset(chart, i, h1)
        action.click()
        action.perform()
        soup = driver.page_source
        soup = BeautifulSoup(soup)
        soup = soup.find_all('tspan')
        results_1 = []
        for i in soup:
            results_1.append(i.find(text = True))
        results = str(results_1[0])
        for i in results_1[1:]:
            results = results + "-" + str(i)
        input_list = [card, results]
        input_list = tuple(input_list)
        print(input_list)
        c.execute('INSERT OR IGNORE INTO card_prices VALUES (?, ?)', input_list)
        conn.commit()

In [ ]:
for i in range(0, 1162):
    card = names[i]
    link = links[i]
    get_data(card, link) 

In [ ]:
conn.close()

In [ ]:
###END###